## Generate **intra**-dataset pairs

In [5]:
import random
from tqdm import tqdm

flag = '_flag_'
def gen_pairs(config_file, data_path):
    db = data_path.split('/')[-1]
    thi_files = os.path.join(data_path, 'thi')
    names = os.listdir(thi_files)
    d = getConfig(config_file)[db]
    prefix_num, postfix, split, trim, session = d.values()
    select_rate = 0.5
    total_len = 0
    genuine = []; imposter = []; imposter_FVC = []
    testi = 0; testj = 0
    for step, i in enumerate(tqdm(names[:-1])):
        if not i.endswith(postfix):
            continue
        name1 = i.replace(postfix, '')
        cur = 0
    #     select1 = np.random.choice(range(1, 7))
    #     select2 = np.random.choice(range(1, 7))
        for ind, j in enumerate(names[step+1:]):
            cur += 1
            if not j.endswith(postfix):
                continue

            name2 = j.replace(postfix, '')
            if db == 'MMCBNU':
                # FIX: 54_2_2 and 54_2_3 in MMCBNU are actually from the same class(genuine matches).
                testi = '_'.join(name1.split('_')[:3]) == '054_2_2'
                testj = '_'.join(name2.split('_')[:3]) == '054_2_3'
            
            tmp = db + '/' + name1 + flag + db + '/' + name2 + flag
            judge = judge_prefix(name1, name2, prefix_num, split)
            if judge or (testi and testj):
                genuine.append(tmp+'1')
            else:
                imposter.append(tmp+'0')
                if not judge_not_FVC(session, prefix_num, name1, name2, step=step):
                    imposter_FVC.append(tmp+'0')
            total_len += 1

    print('totle_len:', total_len, '; genuine:', len(genuine), ':imposter:', len(imposter))
    return total_len, genuine, imposter, imposter_FVC

In [1]:
from utils.utils import *
# names = sorted(names, key=lambda i:str(i.split('.')[0]))

data_path = 'data/FVUSM'
config_file = 'data/config.yaml'
save_dir = 'data/pairs'
os.makedirs(save_dir, exist_ok=True)
# modes = ['full', 'short', 'FVC']
db = data_path.split('/')[-1]
total_len, genuine, imposter, imposter_FVC = gen_pairs(config_file, data_path)

classes = {}
res = []
# pick up all genuine matching
for ii in tqdm(genuine):
    res.append(ii)
gs = len(res)
res_full = res.copy()
res_FVC = res.copy()
res_short = res.copy()
random.shuffle(imposter)
ims = 0

mode = 'FVC'
res = res_FVC
for ii in tqdm(imposter_FVC):
    res_FVC.append(ii)
ims = len(res) - gs
print(db+'-'+mode, f'total: {len(res)}; genuine: {gs}; imposter: {ims}')
random.shuffle(res)
s = '\n'.join(res)
f_new = open(save_dir+'/'+db+'-'+mode+'.txt', 'w')
f_new.write(s)
f_new.close()

mode = 'full'
res = res_full
for ii in tqdm(imposter):
    res.append(ii)
ims = len(res) - gs
print(db+'-'+mode, f'total: {len(res)}; genuine: {gs}; imposter: {ims}')
random.shuffle(res)
s = '\n'.join(res)
f_new = open(save_dir+'/'+db+'-'+mode+'.txt', 'w')
f_new.write(s)
f_new.close()

mode = 'short'
res = res_short
ims = 0
while ims < 10*gs:
    res.append(imposter[ims])
    ims += 1
print(db+'-'+mode, f'total: {len(res)}; genuine: {gs}; imposter: {ims}')
res = res_short
random.shuffle(res)
s = '\n'.join(res)
f_new = open(save_dir+'/'+db+'-'+mode+'.txt', 'w')
f_new.write(s)
f_new.close()

100%|██████████| 32472/32472 [00:00<00:00, 565885.99it/s]


totle_len: 17425656 ; genuine: 32472 :imposter: 17393184


100%|██████████| 120786/120786 [00:00<00:00, 1136877.78it/s]


FVUSM-FVC total: 153258; genuine: 32472; imposter: 120786


100%|██████████| 17393184/17393184 [00:11<00:00, 1484412.25it/s]


FVUSM-full total: 17425656; genuine: 32472; imposter: 17393184
FVUSM-short total: 357192; genuine: 32472; imposter: 324720


## Generate cross/**inter**-dataset pairs

In [4]:
from utils.utils import *
import os

data_root = 'data'
config_file = os.path.join(data_root, 'config.yaml')
cfg = getConfig(config_file)
database = ['HKPU', 'UTFVP', 'MMCBNU', 'SDUMLA', 'FVUSM']

# generate genuine matches
all_genuines = []
for db in database:
    data_path = os.path.join(data_root, db)
    _, genuine, _, _ = gen_pairs(config_file, data_path)
    all_genuines += genuine
# for i in range()

  1%|▏         | 18/1440 [00:00<00:08, 171.46it/s]

totle_len: 4903146 ; genuine: 15390 :imposter: 4887756
HKPU 312


  0%|          | 4/5999 [00:00<03:08, 31.81it/s]

totle_len: 1036080 ; genuine: 2160 :imposter: 1033920
UTFVP 360


  0%|          | 7/3815 [00:00<00:57, 65.84it/s]

totle_len: 17997000 ; genuine: 27050 :imposter: 17969950
MMCBNU 600


  0%|          | 5/5904 [00:00<02:14, 43.90it/s]

totle_len: 7279020 ; genuine: 9540 :imposter: 7269480
SDUMLA 636


100%|██████████| 5904/5904 [01:13<00:00, 80.64it/s] 


totle_len: 17425656 ; genuine: 32472 :imposter: 17393184
FVUSM 492


In [31]:
# collect 1st sample of every class in each dataset
random_sample = 150
database = ['MMCBNU', 'HKPU', 'UTFVP', 'SDUMLA', 'FVUSM']
class_1st_samples = []
for db in database:
    data_path = os.path.join(data_root, db)
    samples = []
    prefix_num, postfix, split, trim, session = cfg[db].values()
    all_files = os.listdir(os.path.join(data_path, 'roi'))
    random.shuffle(all_files)
    for file in all_files:
        if len(samples) >= random_sample:
            break
        if judge_class_1st(session, file.replace(postfix, ''), prefix_num):
            samples.append(db+'/'+file)
    class_1st_samples.append(samples)
    print(db, len(samples))

MMCBNU 150
HKPU 150
UTFVP 150
SDUMLA 150
FVUSM 150


In [32]:
random.shuffle(class_1st_samples)

sample_imposters = []
for ind, db_samples in enumerate(class_1st_samples):
    db1 = db_samples
    db2 = class_1st_samples[(ind+1) % len(class_1st_samples)]
    for i in db1:
        for j in db2:
            sample_imposters.append(i.replace('.bmp', '')+flag+j.replace('.bmp', '')+flag+'0')
print(len(sample_imposters))

112500


In [33]:
with open('data/pairs_meta/cross_dataset.txt', 'w') as f:
    f.write('\n'.join(sample_imposters))
    f.write('\n'.join(all_genuines))
    print('genuines: ', len(all_genuines))
    print('imposters: ', len(sample_imposters))
    print(len(all_genuines)+len(sample_imposters))

genuines:  86612
imposters:  112500
199112
